In [1]:
import pandas as pd
import re
from itertools import permutations
from pulp import *
import numpy as np
import random
from random import choice

from itertools import chain




In [2]:
base = pd.read_csv("combined_data/combined_database_pool.csv")

In [3]:
#dropping useless column
one =base.drop('Game Info', axis =1)

In [4]:
#dropping useless column
two = one.drop('Unnamed: 0', axis =1)

In [5]:
#dropping useless column
three= two.drop('TeamAbbrev', axis=1)


In [6]:
#dropping useless column
four = three.drop('Name + ID', axis=1)

In [7]:

player_arr = four.to_numpy()

In [8]:
qbs= four[four.Position == 'QB']
rbs= four[four.Position == 'RB'].drop('ID', axis =1)
wrs= four[four.Position == 'WR'].drop('ID', axis =1)
tes= four[four.Position == 'TE'].drop('ID', axis =1)
#flex= four[four.Position == 'WR'].drop('ID', axis =1)

#flex = four[four.Position == 'WR' or four[four.Position == 'Rb']
            
flex = four[(four.Position == 'WR') | (four.Position == 'RB')]
defs = four[four.Position == 'DST'].drop('ID', axis =1)

qbs = qbs.drop('ID', axis = 1)

#had to drop ID off Flex DB again
flex_no_id =flex.drop('ID', axis =1)

flex_no_id

,Position,Name,Roster Position,Salary,AvgPointsPerGame
0,RB,Dalvin Cook,RB/FLEX,9400,26.94
1,RB,Christian McCaffrey,RB/FLEX,9200,30.13
2,RB,Derrick Henry,RB/FLEX,8700,20.74
3,RB,Aaron Jones,RB/FLEX,7600,19.93
4,RB,James Robinson,RB/FLEX,7500,19.73
...,...,...,...,...,...
110,WR,Emmanuel Sanders,WR/FLEX,4300,10.76
111,WR,Tim Patrick,WR/FLEX,4200,12.70
112,WR,Darius Slayton,WR/FLEX,4200,10.14
113,WR,Laviska Shenault Jr.,WR/FLEX,4100,9.62


In [9]:
qb_arr = qbs.to_numpy()
wr_arr = wrs.to_numpy()
rb_arr = rbs.to_numpy()
te_arr = tes.to_numpy()
flex_arr =flex_no_id.to_numpy()
def_array = defs.to_numpy()

In [10]:
rb_arr

array([['RB', 'Dalvin Cook', 'RB/FLEX', 9400, 26.94],
       ['RB', 'Christian McCaffrey', 'RB/FLEX', 9200, 30.13],
       ['RB', 'Derrick Henry', 'RB/FLEX', 8700, 20.74],
       ['RB', 'Aaron Jones', 'RB/FLEX', 7600, 19.93],
       ['RB', 'James Robinson', 'RB/FLEX', 7500, 19.73],
       ['RB', 'Alvin Kamara', 'RB/FLEX', 7100, 23.65],
       ['RB', 'Austin Ekeler', 'RB/FLEX', 7000, 16.15],
       ['RB', 'Chris Carson', 'RB/FLEX', 6900, 17.36],
       ['RB', 'Antonio Gibson', 'RB/FLEX', 6700, 15.93],
       ['RB', 'Ezekiel Elliott', 'RB/FLEX', 6600, 16.12],
       ['RB', "D'Andre Swift", 'RB/FLEX', 6500, 14.51],
       ['RB', 'David Montgomery', 'RB/FLEX', 6500, 15.16],
       ['RB', 'Mike Davis', 'RB/FLEX', 6400, 13.69],
       ['RB', 'Alexander Mattison', 'RB/FLEX', 6400, 5.26],
       ['RB', 'Josh Jacobs', 'RB/FLEX', 6300, 16.43],
       ['RB', 'Raheem Mostert', 'RB/FLEX', 6200, 14.15],
       ['RB', 'Miles Sanders', 'RB/FLEX', 6200, 13.04],
       ['RB', 'Ronald Jones II', 'RB/FLEX

In [214]:
#Function that generates a random team of daily fantasy football players.... Salary And duplicates not considered

def team_generator(qb, wr, rb, flex, defense):
    team_list = []
    
    
    
    #Targeting the rows of information on players from each array
    qb_rows = qb.shape[0]
    random_indices = np.random.choice(qb_rows, size=1, replace=False)
    qb_rows = qb[random_indices, :]
    
    wr_rows = wr.shape[0]
    random_indices = np.random.choice(wr_rows, size=3, replace=False)
    wr_rows = wr[random_indices, :]
    
    
    rb_rows = rb.shape[0]
    random_indices = np.random.choice(rb_rows, size=2, replace=False)
    rb_rows = rb[random_indices, :]
    
    flex_rows = flex.shape[0]
    random_indices = np.random.choice(flex_rows, size=1, replace=False)
    flex_rows = flex[random_indices, :]
    
    def_rows = defense.shape[0]
    random_indices = np.random.choice(def_rows, size=1, replace=False)
    def_rows = defense[random_indices, :]
    
    
    
    
    team_list.append(qb_rows)
    team_list.append(wr_rows)
    team_list.append(rb_rows)
    team_list.append(flex_rows)
    team_list.append(def_rows)
    unpack_team = list(chain.from_iterable(team_list))
    
    un_pack_pd = pd.DataFrame(unpack_team)
    
    
    
    
        
    
    
        
    if (un_pack_pd[3].sum() > 50000) or (un_pack_pd[3].sum() <45000):
        
        team_list.clear()
        
        team_list.append(team_generator(qb, wr, rb, flex, defense))
        
        return unpack_team
        
        
        
    #print (un_pack_pd[3].sum())
    
    return unpack_team

random_team = team_generator(qb_arr, wr_arr, rb_arr, flex_arr, def_array)
random_team

[array(['QB', 'Derek Carr', 'QB', 6000, 18.66], dtype=object),
 array(['WR', 'Tyler Boyd', 'WR/FLEX', 4900, 15.6], dtype=object),
 array(['WR', 'Calvin Ridley', 'WR/FLEX', 7500, 19.08], dtype=object),
 array(['WR', 'T.Y. Hilton', 'WR/FLEX', 5100, 9.8], dtype=object),
 array(['RB', 'Boston Scott', 'RB/FLEX', 4900, 6.63], dtype=object),
 array(['RB', 'Devonta Freeman', 'RB/FLEX', 4600, 7.2], dtype=object),
 array(['WR', 'Adam Thielen', 'WR/FLEX', 7000, 19.42], dtype=object),
 array(['DST', 'Seahawks ', 'DST', 3000, 5.83], dtype=object)]

In [172]:
df = pd.DataFrame(random_team)

df

,0,1,2,3,4
0,QB,Drew Brees,QB,6100,19.22
1,WR,Jamison Crowder,WR/FLEX,5400,16.16
2,WR,A.J. Brown,WR/FLEX,7300,17.55
3,WR,Robby Anderson,WR/FLEX,6200,15.71
4,RB,David Montgomery,RB/FLEX,6500,15.16
5,RB,Todd Gurley II,RB/FLEX,4800,13.20
6,WR,Corey Davis,WR/FLEX,5700,16.91
7,DST,Vikings,DST,2300,4.83


In [211]:
#function that generates 1000 different samples

def sample_generator(samples = 1000):

    sample_list = []
    
    
    
    for _ in range(samples):
        sample_list.append(team_generator(qb_arr, wr_arr, rb_arr, flex_arr, def_array))
        
    unpack_samples = list(chain.from_iterable(sample_list))
    
    convert_to_pd = pd.DataFrame(unpack_samples)
    
    convert_to_pd.columns = ['pos', 'name', 'fant_pos', 'salary', 'proj_point']
    
   
    
    return convert_to_pd


x = sample_generator(10)


#team_samps, salary_list  = sample_generator(10)

y =  x['salary'].rolling(8).sum().shift(-8)
y

46500
47900
45000
45900
46300
47000
48000
45600
46300
47100


0     46100.0
1     48300.0
2     46900.0
3     46300.0
4     46900.0
       ...   
75        NaN
76        NaN
77        NaN
78        NaN
79        NaN
Name: salary, Length: 80, dtype: float64

0     41300.0
1     43000.0
2     44000.0
3     43400.0
4     44800.0
       ...   
59    41000.0
60    41300.0
61    39900.0
62    39800.0
63    40700.0
Name: salary, Length: 64, dtype: float64

In [174]:
team_samps.index = team_samps.index // 8



In [175]:
team_samps = team_samps.reset_index()

In [176]:
team_samps

,index,pos,name,fant_pos,salary,proj_point
0,0,QB,Tom Brady,QB,6900,21.89
1,0,WR,Henry Ruggs III,WR/FLEX,4600,7.78
2,0,WR,Chris Godwin,WR/FLEX,6300,15.40
3,0,WR,Mike Williams,WR/FLEX,4700,10.94
4,0,RB,Salvon Ahmed,RB/FLEX,4800,10.73
...,...,...,...,...,...,...
75,9,WR,Sterling Shepard,WR/FLEX,5200,11.36
76,9,RB,Duke Johnson,RB/FLEX,4800,7.75
77,9,RB,Aaron Jones,RB/FLEX,7600,19.93
78,9,RB,Nyheim Hines,RB/FLEX,5200,12.60


In [177]:
team_samps

,index,pos,name,fant_pos,salary,proj_point
0,0,QB,Tom Brady,QB,6900,21.89
1,0,WR,Henry Ruggs III,WR/FLEX,4600,7.78
2,0,WR,Chris Godwin,WR/FLEX,6300,15.40
3,0,WR,Mike Williams,WR/FLEX,4700,10.94
4,0,RB,Salvon Ahmed,RB/FLEX,4800,10.73
...,...,...,...,...,...,...
75,9,WR,Sterling Shepard,WR/FLEX,5200,11.36
76,9,RB,Duke Johnson,RB/FLEX,4800,7.75
77,9,RB,Aaron Jones,RB/FLEX,7600,19.93
78,9,RB,Nyheim Hines,RB/FLEX,5200,12.60


In [180]:
y = team_samps.groupby("index")["salary"].agg("sum")



In [181]:
y

index
0    42600
1    45000
2    39600
3    47000
4    45700
5    46300
6    41500
7    49100
8    41900
9    44300
Name: salary, dtype: int64